In [1]:
pip install face_recognition

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import cv2
import face_recognition
import os
import glob
import numpy as np


In [2]:
img = cv2.imread(r'C:\Users\seifm\Downloads\aboutrika.jpg')
rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_encoding = face_recognition.face_encodings(rgb_img)[0]

In [3]:
img2 = cv2.imread(r'C:\Users\seifm\Downloads\Aboutrika2011.jpg')
rgb_img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
img_encoding2 = face_recognition.face_encodings(rgb_img2)[0]

In [4]:
result = face_recognition.compare_faces([img_encoding], img_encoding2)
print("Result: ", result)

Result:  [True]


In [2]:
class SimpleFacerec:
    def __init__(self):
        self.known_face_encodings = []
        self.known_face_names = []

        # Resize frame for a faster speed
        self.frame_resizing = 0.25

    def load_encoding_images(self, images_path):
        """
        Load encoding images from path
        :param images_path:
        :return:
        """
        # Load Images
        images_path = glob.glob(os.path.join(images_path, "*.*"))

        print("{} encoding images found.".format(len(images_path)))

        # Store image encoding and names
        for img_path in images_path:
            img = cv2.imread(img_path)
            rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Get the filename only from the initial file path.
            basename = os.path.basename(img_path)
            (filename, ext) = os.path.splitext(basename)
            # Get encoding
            img_encoding = face_recognition.face_encodings(rgb_img)[0]

            # Store file name and file encoding
            self.known_face_encodings.append(img_encoding)
            self.known_face_names.append(filename)
        print("Encoding images loaded")

    def detect_known_faces(self, frame):
        small_frame = cv2.resize(frame, (0, 0), fx=self.frame_resizing, fy=self.frame_resizing)
        # Find all the faces and face encodings in the current frame of video
        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(self.known_face_encodings, face_encoding)
            name = "Unknown"

            # # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = self.known_face_names[best_match_index]
            face_names.append(name)

        # Convert to numpy array to adjust coordinates with frame resizing quickly
        face_locations = np.array(face_locations)
        face_locations = face_locations / self.frame_resizing
        return face_locations.astype(int), face_names


In [4]:
sfr = SimpleFacerec()
sfr.load_encoding_images(r"E:\Downloads\source-code-face-recognition\source code\images")
cap = cv2.VideoCapture(1)
while cap.isOpened():
        
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
        # If loading a video, use 'break' instead of 'continue'.
      continue
    image.flags.writeable = False
# Load Camera
    # Detect Faces
    face_locations, face_names = sfr.detect_known_faces(image)
    for face_loc, name in zip(face_locations, face_names):
        y1, x2, y2, x1 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]

        cv2.putText(image, name,(x1, y1 - 10), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 200), 2)
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 200), 4)

    cv2.imshow("Frame", image)

    #key = cv2.waitKey(1)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

7 encoding images found.
Encoding images loaded


In [15]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
mp_holistic = mp.solutions.holistic # Mediapipe Solutions
mp_face_mesh = mp.solutions.face_mesh

In [16]:
data=""
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(
        static_image_mode=True, min_detection_confidence=0.5, model_complexity=2) as holistic:
    while cap.isOpened():
     success, image = cap.read()
     if not success:
      print("Ignoring empty camera frame.")
        # If loading a video, use 'break' instead of 'continue'.
      continue
     image.flags.writeable = False
     results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        # Print nose coordinates.
     image_hight, image_width, _ = image.shape
     if results.pose_landmarks:
          print(
            f'Nose coordinates: ('
            f'{results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].x * image_width}, '
            f'{results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].y * image_hight})'
          )

        # Draw pose landmarks.
     print(f'Pose landmarks :')
     annotated_image = image.copy()
     mp_drawing.draw_landmarks(
            annotated_image, 
            results.left_hand_landmarks, 
            mp_holistic.HAND_CONNECTIONS)
     mp_drawing.draw_landmarks(
            annotated_image, 
            results.right_hand_landmarks, 
            mp_holistic.HAND_CONNECTIONS)
     mp_drawing.draw_landmarks(
            annotated_image,
            results.face_landmarks,
            mp_holistic.FACEMESH_TESSELATION,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_tesselation_style())
     mp_drawing.draw_landmarks(
            annotated_image,
            results.pose_landmarks,
            mp_holistic.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.
            get_default_pose_landmarks_style())
        #viewImage(annotated_image)
        #return results.pose_landmarks
     #data+=str(face_landmarks.landmark[point].x*image_width)+","+str(face_landmarks.landmark[point].y*image_height)+","
     #print(data)
    # Flip the image horizontally for a selfie-view display.
     cv2.imshow('MediaPipe Face Mesh', cv2.flip(annotated_image, 1))
     if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
cap.release()
cv2.destroyAllWindows()

Nose coordinates: (374.4115447998047, 315.1494026184082)
Pose landmarks :
Nose coordinates: (381.10260009765625, 316.695556640625)
Pose landmarks :
Nose coordinates: (354.1220474243164, 301.1458969116211)
Pose landmarks :
Nose coordinates: (351.6651916503906, 300.75465202331543)
Pose landmarks :
Nose coordinates: (343.98143768310547, 299.6862602233887)
Pose landmarks :
Nose coordinates: (348.8886260986328, 298.4597969055176)
Pose landmarks :
Nose coordinates: (349.6786880493164, 281.85619354248047)
Pose landmarks :
Nose coordinates: (375.32474517822266, 277.55773544311523)
Pose landmarks :
Pose landmarks :
Pose landmarks :
Pose landmarks :
Pose landmarks :
Pose landmarks :
Pose landmarks :
Pose landmarks :
Pose landmarks :
Pose landmarks :
Pose landmarks :
Pose landmarks :
Pose landmarks :
Pose landmarks :
Pose landmarks :
Nose coordinates: (346.6661834716797, 229.99590396881104)
Pose landmarks :
Pose landmarks :
Pose landmarks :
Pose landmarks :
Pose landmarks :
Nose coordinates: (372

In [25]:
import bluetooth
nearby_devices = bluetooth.discover_devices(lookup_names=True)
print("found %d devices" % len(nearby_devices))

for addr, name in nearby_devices:
     print(" %s - %s" % (addr, name))
     

found 0 devices


In [ ]:
import socket
mySocket = socket.socket()
mySocket.bind(('localhost', 5000))
mySocket.listen(5)
conn , addr = mySocket.accept()
print("device connected")
# print(conn,addr)
msg =bytes(name,'UTF-8')
# print(msg)
conn.send(msg)